In [1]:
import os
import pandas as pd
import yfinance as yf
from fredapi import Fred
import stockstats

In [2]:
ticker = "AAPL"
period = "5y"
start = "2017-01-01"
end = "2022-12-31"

In [3]:
os.environ["FRED_KEY"]

'895660a68c8642ce5ea73a8e7f9273b4'

In [4]:
fred = Fred(api_key=os.environ["FRED_KEY"])
data = fred.get_series('EFFR')

In [5]:
interest_rate_df = fred.get_series('EFFR')
unemployment_rate_df = fred.get_series('UNRATE')
consumer_sent_df = fred.get_series('UMCSENT')

interest_rate_df.index = interest_rate_df.index.strftime("%Y-%m-%d")
unemployment_rate_df.index = unemployment_rate_df.index.strftime("%Y-%m-%d")
consumer_sent_df.index = consumer_sent_df.index.strftime("%Y-%m-%d")

interest_rate_df.name = "EFFR"
unemployment_rate_df.name = "UNRATE"
consumer_sent_df.name = "UMCSENT"

In [6]:
combined_df = pd.merge(interest_rate_df, unemployment_rate_df, left_index=True, right_index=True, how="outer")
combined_df = pd.merge(combined_df, consumer_sent_df, left_index=True, right_index=True, how="outer")

In [7]:
combined_df.index.name = "Date"

In [8]:
usd = yf.Ticker("DX-Y.NYB")
usd_df = usd.history(start=start, end=end)
usd_df.index = usd_df.index.strftime("%Y-%m-%d")
usd_df.columns = usd_df.columns.str.lower()
usd_df.columns = [i + "_usd" for i in usd_df.columns]

In [9]:
vix = yf.Ticker("^VIX")
vix_df = vix.history(start=start, end=end)
vix_df.index = vix_df.index.strftime("%Y-%m-%d")
vix_df.columns = vix_df.columns.str.lower()

In [10]:
t = yf.Ticker(ticker)
df = t.history(start=start, end=end)
df.index = df.index.strftime("%Y-%m-%d")
df.columns = df.columns.str.lower()

In [11]:
df

,open,high,low,close,volume,dividends,stock splits
Date,,,,,,,
2017-01-03,27.014199,27.137839,26.771585,27.095848,115127600,0.0,0.0
2017-01-04,27.025862,27.179829,27.002534,27.065519,84472400,0.0,0.0
2017-01-05,27.042193,27.261480,27.016532,27.203159,88774400,0.0,0.0
2017-01-06,27.242817,27.564749,27.170499,27.506428,127007600,0.0,0.0
2017-01-09,27.515753,27.861013,27.513421,27.758368,134247600,0.0,0.0
...,...,...,...,...,...,...,...
2022-12-23,130.539854,132.035499,129.263572,131.477127,63814900,0.0,0.0
2022-12-27,130.998521,131.028433,128.346242,129.652435,69007800,0.0,0.0
2022-12-28,129.293476,130.649527,125.504514,125.674019,85438400,0.0,0.0


In [12]:
ss_df = stockstats.wrap(df)
ss_df[['macd', 'rsi', 'atr']];

In [13]:
yf_combined_df = ss_df.merge(vix_df, on="Date", how="outer", suffixes = (f"_{ticker}", "_vix"))
yf_combined_df = yf_combined_df.merge(usd_df, on="Date", how="outer", suffixes=("", "_usd"))

In [14]:
yf_combined_df

,open_AAPL,high_AAPL,low_AAPL,close_AAPL,volume_AAPL,dividends_AAPL,stock splits_AAPL,macd,macds,macdh,...,volume_vix,dividends_vix,stock splits_vix,open_usd,high_usd,low_usd,close_usd,volume_usd,dividends_usd,stock splits_usd
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,27.014199,27.137839,26.771585,27.095848,115127600,0.0,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,102.870003,103.820000,102.589996,103.209999,0,0.0,0.0
2017-01-04,27.025862,27.179829,27.002534,27.065519,84472400,0.0,0.0,-0.000680,-0.000378,-0.000302,...,0,0.0,0.0,103.180000,103.440002,102.389999,102.699997,0,0.0,0.0
2017-01-05,27.042193,27.261480,27.016532,27.203159,88774400,0.0,0.0,0.003401,0.001171,0.002230,...,0,0.0,0.0,102.430000,102.510002,101.300003,101.519997,0,0.0,0.0
2017-01-06,27.242817,27.564749,27.170499,27.506428,127007600,0.0,0.0,0.016124,0.006236,0.009887,...,0,0.0,0.0,101.419998,102.290001,101.370003,102.220001,0,0.0,0.0
2017-01-09,27.515753,27.861013,27.513421,27.758368,134247600,0.0,0.0,0.032723,0.014115,0.018607,...,0,0.0,0.0,102.260002,102.519997,101.800003,101.930000,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,130.539854,132.035499,129.263572,131.477127,63814900,0.0,0.0,-3.644183,-2.540708,-1.103475,...,0,0.0,0.0,104.389999,104.540001,104.120003,104.309998,0,0.0,0.0
2022-12-27,130.998521,131.028433,128.346242,129.652435,69007800,0.0,0.0,-3.910067,-2.814580,-1.095487,...,0,0.0,0.0,104.120003,104.400002,103.889999,104.180000,0,0.0,0.0
2022-12-28,129.293476,130.649527,125.504514,125.674019,85438400,0.0,0.0,-4.391188,-3.129901,-1.261286,...,0,0.0,0.0,104.330002,104.559998,103.849998,104.459999,0,0.0,0.0


In [15]:
combined_df = combined_df.merge(yf_combined_df, on="Date", how="outer")

In [17]:
combined_df

,EFFR,UNRATE,UMCSENT,open_AAPL,high_AAPL,low_AAPL,close_AAPL,volume_AAPL,dividends_AAPL,stock splits_AAPL,...,volume_vix,dividends_vix,stock splits_vix,open_usd,high_usd,low_usd,close_usd,volume_usd,dividends_usd,stock splits_usd
Date,,,,,,,,,,,,,,,,,,,,,
1948-01-01,NaN,3.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948-02-01,NaN,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948-03-01,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948-04-01,NaN,3.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948-05-01,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-25,5.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-26,5.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-27,5.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
combined_df["EFFR"] = combined_df["EFFR"].ffill()
combined_df["UNRATE"] = combined_df["UNRATE"].ffill()
combined_df["UMCSENT"] = combined_df["UMCSENT"].ffill()

In [22]:
final_data_df = combined_df.dropna()

In [24]:
final_data_df.to_csv(f"data_{ticker}_{start}_{end}.csv")